In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
# =========================
# 1. IMPORTS
# =========================
import torch
import torch.nn as nn
import torch.nn.functional as F

# =========================
# 2. STATIC Q&A TRAINING DATA
# =========================
text = """
User: What is Artificial Intelligence?
Assistant: Artificial Intelligence is the ability of machines to mimic human intelligence such as learning, reasoning, and decision making.

User: What is Generative AI?
Assistant: Generative AI refers to models that can generate new data such as text, images, audio, and code.

User: Why is AI important in cybersecurity?
Assistant: AI is important in cybersecurity because it helps detect threats, automate responses, and defend against cyber attacks faster than humans.

User: What is machine learning?
Assistant: Machine learning is a subset of artificial intelligence that allows systems to learn patterns from data without being explicitly programmed.
"""

# =========================
# 3. TOKENIZATION (CHAR LEVEL)
# =========================
chars = sorted(list(set(text)))
stoi = {ch: i for i, ch in enumerate(chars)}
itos = {i: ch for ch, i in stoi.items()}
vocab_size = len(chars)

data = torch.tensor([stoi[c] for c in text], dtype=torch.long)

# =========================
# 4. GPT-LIKE MINI TRANSFORMER MODEL
# =========================
class MiniGPT(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()

        self.token_emb = nn.Embedding(vocab_size, 64)
        self.pos_emb = nn.Embedding(2048, 64)   # SAFE SIZE

        encoder_layer = nn.TransformerEncoderLayer(
            d_model=64,
            nhead=4,
            batch_first=True
        )
        self.transformer = nn.TransformerEncoder(encoder_layer, num_layers=2)
        self.fc = nn.Linear(64, vocab_size)

    def forward(self, x):
        seq_len = x.size(1)
        positions = torch.arange(seq_len, device=x.device).unsqueeze(0)

        x = self.token_emb(x) + self.pos_emb(positions)
        x = self.transformer(x)
        return self.fc(x)

model = MiniGPT(vocab_size)

# =========================
# 5. TRAINING LOOP (REQUIRED)
# =========================
optimizer = torch.optim.Adam(model.parameters(), lr=0.003)
loss_fn = nn.CrossEntropyLoss()

for epoch in range(300):
    optimizer.zero_grad()

    x = data[:-1].unsqueeze(0)
    y = data[1:].unsqueeze(0)

    logits = model(x)
    loss = loss_fn(logits.view(-1, vocab_size), y.view(-1))

    loss.backward()
    optimizer.step()

    if epoch % 50 == 0:
        print(f"Epoch {epoch} | Loss: {loss.item():.4f}")

# =========================
# 6. TEXT GENERATION FUNCTION
# =========================
def generate_text(prompt, max_len=300):
    model.eval()
    tokens = torch.tensor([[stoi[c] for c in prompt]], dtype=torch.long)

    for _ in range(max_len):
        logits = model(tokens)
        next_token = torch.argmax(logits[:, -1, :], dim=-1)
        tokens = torch.cat([tokens, next_token.unsqueeze(1)], dim=1)

    return ''.join([itos[i.item()] for i in tokens[0]])

# =========================
# 7. ASK QUESTION (CHATGPT STYLE)
# =========================
question = "User: What is Generative AI?\nAssistant:"
response = generate_text(question)

print("\n===== AI RESPONSE =====\n")
print(response)

Epoch 0 | Loss: 3.7250
Epoch 50 | Loss: 0.0359
Epoch 100 | Loss: 0.0048
Epoch 150 | Loss: 0.0028
Epoch 200 | Loss: 0.0018
Epoch 250 | Loss: 0.0013

===== AI RESPONSE =====

User: What is Generative AI?
Assistant: Assistant: Artificial Intelligence is the ability of machines to mimic human intelligence such as learning, reasoning, and decision making.

User: What is Generative AI?
Assistant: Generative AI refers to models that can generate new data such as text, images, audio, and code.

User: Why is AI impo
